In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer 
import numpy
import nltk, re, string, collections
from nltk.util import ngrams

In [2]:
def read_cleaned_data_set():
    file = open("output/tokens_no_stopwords_train.txt", encoding='utf-8')
    corpus =[]
    lines = file.readlines()
    for line in lines:
        result=line.rstrip('\n')
        if len(result)>0:
            result = result[1:len(result)-2]
            result=result.replace(",","")
            result= result.replace("'","")
        
            corpus.append(result)
    return corpus

In [3]:
corpus =read_cleaned_data_set()


In [4]:
def get_TF_IDF(docs): 
    #create tf-idf vector & enable idf to calculate them in one steps
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    #get values for docs
    tfIdf = tfIdfVectorizer.fit_transform(docs)
    # get first tf-idfs for first doc only to be visualized
    # get features names mean get words
    df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    #sorting them
    df = df.sort_values('TF-IDF', ascending=False)
    #show first 15 of them 
    print (df.head(15))
    


In [5]:
get_TF_IDF(corpus)

            TF-IDF
صيفي      0.243324
السيرنجة  0.243324
مزايا     0.243324
بولو      0.243324
اصغر      0.232552
مؤهل      0.232552
الشتاء    0.224910
لابس      0.224910
تصوير     0.218981
الابرة    0.210042
يعنى      0.206495
غيتس      0.203366
عز        0.198035
بيل       0.195723
احدى      0.191628


c:\Users\rorod\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
def get_tokens_freq(corpus):
    # corpus_new = corpus.split()
    tokens ={}
    for text in corpus:
        text = text.split()
        for value in text:
            if value in tokens :
                tokens[value]+=1
            else:
                tokens[value]=1
    return tokens

In [32]:
def get_n_grams(corpus, n_gram):
    all_n_grams=[]
    for text in corpus:
        padding = ["باد"]*(n_gram-1)
        text = text.split()
        text = padding+ text+padding
        n_grams = ngrams(text, n_gram)
        all_n_grams+=n_grams
    ngramFreq = collections.Counter(all_n_grams)
    return ngramFreq


In [37]:
def convert_ngrams_probablities( n,corpus):
  probablities =[]
  ngrams= get_n_grams(corpus,n)
  
  # print("kdkdk",ngrams)
  if n>1:
    n_minus_one_grams = get_n_grams(corpus, n-1)
  tokens =get_tokens_freq(corpus)
  for text in corpus:
    padding = ["باد"]*(n-1)
    text_value=text.split()
    text_value= padding+text_value+padding
    prob_list= []
    for index in range(n, len(text_value)-n) :
      n_gram_text = tuple(text_value[index-n:index ])
      n_minus_one_grams_text = tuple(text_value[index-n:index-1])
      #here to add one smooth padding
      # nomiantor=1
      # dominator=len(tokens) 
      nomiantor=0.0
      dominator=0.0
      if n_gram_text in ngrams:
  
        nomiantor +=ngrams[n_gram_text]
      if n_minus_one_grams_text in n_minus_one_grams:
  
        dominator+= n_minus_one_grams[n_minus_one_grams_text]

      elif n==1:
        dominator += tokens[n_minus_one_grams_text]
      prob=nomiantor /dominator
      prob_list.append(prob) # for one doc
    probablities.append(prob_list) # for all docs
    
  return probablities
    

In [38]:
prob=convert_ngrams_probablities(3,corpus)
result=get_n_grams(corpus,2)
# print(result)
# # result.
result.most_common(10)


[(('لقاح', 'كورونا'), 1369),
 (('الجرعة', 'الاولى'), 665),
 (('الاولى', 'لقاح'), 574),
 (('كورونا', 'باد'), 469),
 (('يتلقى', 'الجرعة'), 424),
 (('جرعة', 'لقاح'), 400),
 (('لقاح', 'باد'), 385),
 (('لقاح', 'فايزر'), 333),
 (('فيروس', 'كورونا'), 313),
 (('الجرعة', 'الثانية'), 308)]

In [39]:
print((prob[0]))

[7.0, 0.5714285714285714, 0.2857142857142857, 0.5, 0.007874015748031496, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.047619047619047616, 1.0, 1.0, 1.0, 1.0, 0.09090909090909091, 1.0, 1.0, 0.5, 0.3333333333333333, 1.0, 1.0]


In [40]:
print(prob[100])

[1.0, 1.0, 1.0, 0.25, 1.0, 0.125, 0.03003003003003003, 0.06666666666666667, 1.0, 1.0, 1.0, 0.1111111111111111, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
